# Data Loading and Preprocessing

# Processes the data from the CSV file into a pandas dataframe ready for analysis.

In [1]:
import pandas as pd

main = pd.read_csv('../data/emails.csv')
main

,file,message
0,allen-p/_sent_mail/1.,Message-ID: <18782981.1075855378110.JavaMail.e...
1,allen-p/_sent_mail/10.,Message-ID: <15464986.1075855378456.JavaMail.e...
2,allen-p/_sent_mail/100.,Message-ID: <24216240.1075855687451.JavaMail.e...
3,allen-p/_sent_mail/1000.,Message-ID: <13505866.1075863688222.JavaMail.e...
4,allen-p/_sent_mail/1001.,Message-ID: <30922949.1075863688243.JavaMail.e...
...,...,...
517396,zufferli-j/sent_items/95.,Message-ID: <26807948.1075842029936.JavaMail.e...
517397,zufferli-j/sent_items/96.,Message-ID: <25835861.1075842029959.JavaMail.e...
517398,zufferli-j/sent_items/97.,Message-ID: <28979867.1075842029988.JavaMail.e...
517399,zufferli-j/sent_items/98.,Message-ID: <22052556.1075842030013.JavaMail.e...


In [2]:
main['message'][0]

"Message-ID: <18782981.1075855378110.JavaMail.evans@thyme>\nDate: Mon, 14 May 2001 16:39:00 -0700 (PDT)\nFrom: phillip.allen@enron.com\nTo: tim.belden@enron.com\nSubject: \nMime-Version: 1.0\nContent-Type: text/plain; charset=us-ascii\nContent-Transfer-Encoding: 7bit\nX-From: Phillip K Allen\nX-To: Tim Belden <Tim Belden/Enron@EnronXGate>\nX-cc: \nX-bcc: \nX-Folder: \\Phillip_Allen_Jan2002_1\\Allen, Phillip K.\\'Sent Mail\nX-Origin: Allen-P\nX-FileName: pallen (Non-Privileged).pst\n\nHere is our forecast\n\n "

In [3]:
def get_message(Series: pd.Series):
    result = pd.Series(index=Series.index)
    for row, message in enumerate(Series):
        message_words = message.split('\n')
        del message_words[:15]
        result.iloc[row] = ''.join(message_words).strip()
    return result

def get_date(Series: pd.Series):
    result = pd.Series(index=Series.index)
    for row, message in enumerate(Series):
        message_words = message.split('\n')
        del message_words[0]
        del message_words[1:]
        result.iloc[row] = ''.join(message_words).strip()
        result.iloc[row] = result.iloc[row].replace('Date: ', '')
    print('Done parsing, converting to datetime format..')
    return pd.to_datetime(result)

def get_sender_and_receiver(Series: pd.Series):
    sender = pd.Series(index = Series.index)
    recipient1 = pd.Series(index = Series.index)
    recipient2 = pd.Series(index = Series.index)
    recipient3 = pd.Series(index = Series.index)
    
    for row,message in enumerate(Series):
        message_words = message.split('\n')
        sender[row] = message_words[2].replace('From: ', '')
        recipient1[row] = message_words[3].replace('To: ', '')
        recipient2[row] = message_words[10].replace('X-cc: ', '')
        recipient3[row] = message_words[11].replace('X-bcc: ', '')
        
    return sender, recipient1, recipient2, recipient3


def get_subject(Series: pd.Series):
    result = pd.Series(index = Series.index)
    
    for row, message in enumerate(Series):
        message_words = message.split('\n')
        message_words = message_words[4]
        result[row] = message_words.replace('Subject: ', '')
    return result

def get_folder(Series: pd.Series):
    result = pd.Series(index = Series.index)
    
    for row, message in enumerate(Series):
        message_words = message.split('\n')
        message_words = message_words[12]
        result[row] = message_words.replace('X-Folder: ', '')
    return result



In [4]:
main['text'] = get_message(main.message)
main['sender'], main['recipient1'], main['recipient2'], main['recipient3'] = get_sender_and_receiver(main.message)
main['Subject'] = get_subject(main.message)
main['folder'] = get_folder(main.message)
main['date'] = get_date(main.message)



/var/folders/42/5n545hgx2_g9xwrm2dmpjwjr0000gp/T/ipykernel_8613/2675128756.py:6: FutureWarning: Setting an item of incompatible dtype is deprecated and will raise an error in a future version of pandas. Value 'Here is our forecast' has dtype incompatible with float64, please explicitly cast to a compatible dtype first.
  result.iloc[row] = ''.join(message_words).strip()
/var/folders/42/5n545hgx2_g9xwrm2dmpjwjr0000gp/T/ipykernel_8613/2675128756.py:28: FutureWarning: Setting an item of incompatible dtype is deprecated and will raise an error in a future version of pandas. Value 'phillip.allen@enron.com' has dtype incompatible with float64, please explicitly cast to a compatible dtype first.
  sender[row] = message_words[2].replace('From: ', '')
/var/folders/42/5n545hgx2_g9xwrm2dmpjwjr0000gp/T/ipykernel_8613/2675128756.py:29: FutureWarning: Setting an item of incompatible dtype is deprecated and will raise an error in a future version of pandas. Value 'tim.belden@enron.com' has dtype inco

Done parsing, converting to datetime format..


/var/folders/42/5n545hgx2_g9xwrm2dmpjwjr0000gp/T/ipykernel_8613/2675128756.py:18: UserWarning: Could not infer format, so each element will be parsed individually, falling back to `dateutil`. To ensure parsing is consistent and as-expected, please specify a format.
  return pd.to_datetime(result)
/var/folders/42/5n545hgx2_g9xwrm2dmpjwjr0000gp/T/ipykernel_8613/2675128756.py:18: FutureWarning: In a future version of pandas, parsing datetimes with mixed time zones will raise an error unless `utc=True`. Please specify `utc=True` to opt in to the new behaviour and silence this warning. To create a `Series` with mixed offsets and `object` dtype, please use `apply` and `datetime.datetime.strptime`
  return pd.to_datetime(result)


In [5]:
main

,file,message,text,sender,recipient1,recipient2,recipient3,Subject,folder,date
0,allen-p/_sent_mail/1.,Message-ID: <18782981.1075855378110.JavaMail.e...,Here is our forecast,phillip.allen@enron.com,tim.belden@enron.com,,,,"\Phillip_Allen_Jan2002_1\Allen, Phillip K.\'Se...",2001-05-14 16:39:00-07:00
1,allen-p/_sent_mail/10.,Message-ID: <15464986.1075855378456.JavaMail.e...,Traveling to have a business meeting takes the...,phillip.allen@enron.com,john.lavorato@enron.com,,,Re:,"\Phillip_Allen_Jan2002_1\Allen, Phillip K.\'Se...",2001-05-04 13:51:00-07:00
2,allen-p/_sent_mail/100.,Message-ID: <24216240.1075855687451.JavaMail.e...,test successful. way to go!!!,phillip.allen@enron.com,leah.arsdall@enron.com,,,Re: test,\Phillip_Allen_Dec2000\Notes Folders\'sent mail,2000-10-18 03:00:00-07:00
3,allen-p/_sent_mail/1000.,Message-ID: <13505866.1075863688222.JavaMail.e...,"Randy, Can you send me a schedule of the salar...",phillip.allen@enron.com,randall.gay@enron.com,,,,\Phillip_Allen_Dec2000\Notes Folders\'sent mail,2000-10-23 06:13:00-07:00
4,allen-p/_sent_mail/1001.,Message-ID: <30922949.1075863688243.JavaMail.e...,Let's shoot for Tuesday at 11:45.,phillip.allen@enron.com,greg.piper@enron.com,,,Re: Hello,\Phillip_Allen_Dec2000\Notes Folders\'sent mail,2000-08-31 05:07:00-07:00
...,...,...,...,...,...,...,...,...,...,...
517396,zufferli-j/sent_items/95.,Message-ID: <26807948.1075842029936.JavaMail.e...,This is a trade with OIL-SPEC-HEDGE-NG (John L...,john.zufferli@enron.com,kori.loibl@enron.com,,,Trade with John Lavorato,"\ExMerge - Zufferli, John\Sent Items",2001-11-28 13:30:11-08:00
517397,zufferli-j/sent_items/96.,Message-ID: <25835861.1075842029959.JavaMail.e...,Some of my position is with the Alberta Term b...,john.zufferli@enron.com,john.lavorato@enron.com,,,Gas Hedges,"\ExMerge - Zufferli, John\Sent Items",2001-11-28 12:47:48-08:00
517398,zufferli-j/sent_items/97.,Message-ID: <28979867.1075842029988.JavaMail.e...,"2 -----Original Message-----From: \tDoucet, Da...",john.zufferli@enron.com,dawn.doucet@enron.com,,,RE: CONFIDENTIAL,"\ExMerge - Zufferli, John\Sent Items",2001-11-28 07:20:00-08:00
517399,zufferli-j/sent_items/98.,Message-ID: <22052556.1075842030013.JavaMail.e...,Analyst\t\t\t\t\tRankStephane Brodeur\t\t\t1Ch...,john.zufferli@enron.com,jeanie.slone@enron.com,,,Calgary Analyst/Associate,"\ExMerge - Zufferli, John\Sent Items",2001-11-27 11:52:45-08:00


In [6]:
main.to_csv('../data/processed_emails.csv',index=False)